In [ ]:
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image

import code
import matplotlib.pyplot as plt

import tensorflow.python.platform
import sys
import numpy
import tensorflow as tf
import argparse
import numpy as np
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.layers import Conv2D
from scipy.ndimage import rotate
from helpers import *
from constants import *
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, losses, callbacks
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

%load_ext autoreload
%autoreload 2



In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


# Data extraction and splitting

In [ ]:
data_dir = '/content/drive/MyDrive/Colab Notebooks/datasets/training/'
data_filename = data_dir + "images/"
labels_filename = data_dir + "groundtruth/"


data, labels = load_data(data_filename, labels_filename, TRAINING_SIZE)


Loading 100 images
[[0.3254902  0.3019608  0.27058825]
 [0.31764707 0.28627452 0.25490198]
 [0.3137255  0.2901961  0.25490198]
 ...
 [0.31764707 0.31764707 0.29411766]
 [0.3137255  0.3137255  0.2901961 ]
 [0.31764707 0.31764707 0.3019608 ]]
Loading 100 images


In [7]:
# data splitting
train_split = 0.7
val_split = 0.15
test_split = 0.15

train_data = data[:int(train_split * len(data))]
train_labels = labels[:int(train_split * len(labels))]
val_data = data[int(train_split * len(data)) : int((train_split + val_split) * len(data))]
val_labels = labels[int(train_split * len(labels)) : int((train_split + val_split) * len(labels))]
test_data = data[int((train_split + val_split) * len(data)):]
test_labels = labels[int((train_split + val_split) * len(labels)):]


# Model definition and training

In [ ]:

def residual_block(x, filters, kernel_size=(3, 3)):
    """Defines a residual block with two convolutional layers."""
    shortcut = x
    # Project shortcut to the same shape if needed
    if x.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), padding='same',
                                 kernel_regularizer=regularizers.l2(0.01))(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    # Convolutional path
    x = layers.Conv2D(filters, kernel_size, padding='same', activation='relu',
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, kernel_size, padding='same', activation=None,
                      kernel_regularizer=regularizers.l2(0.01))(x)
    x = layers.BatchNormalization()(x)

    # Add shortcut and convolutions
    x = layers.add([shortcut, x])
    x = layers.Activation('relu')(x)
    return x


# Input dimensions
input_shape = (100, 100, 3)  # Increased input size for more spatial context
num_classes = 2  # Binary segmentation: road or not-road (single channel output)

# Model definition
inputs = layers.Input(shape=input_shape)

# First convolutional block
x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling2D((2, 2))(x)
# Residual blocks
x = residual_block(x, 64)
x = layers.MaxPooling2D((2, 2))(x)

x = residual_block(x, 128)
x = layers.MaxPooling2D((2, 2))(x)

x = residual_block(x, 256)
x = layers.MaxPooling2D((2, 2))(x)

# Bottleneck
x = layers.Conv2D(512, (3, 3), padding='same', activation='relu',
                   kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.BatchNormalization()(x)
x = layers.GlobalAveragePooling2D()(x)

# Fully connected layers
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = models.Model(inputs, outputs)

# Model summary
#model.summary()




In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
def create_generators(train_data, train_labels, val_data, val_labels, test_data, test_labels, WINDOW_SIZE, batch_size=32):
    c_weights = {0: 2.8, 1: 1}
    train_generator = image_generator_old(
        train_data,
        train_labels,
        WINDOW_SIZE,
        batch_size=batch_size,
        class_weights = c_weights)

    val_generator = image_generator_old(
        val_data,
        val_labels,
        WINDOW_SIZE,
        batch_size=batch_size)

    test_generator = image_generator_old(
        test_data,
        test_labels,
        WINDOW_SIZE,
        batch_size=batch_size)

    return train_generator, val_generator, test_generator


In [30]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate= 0.0005),
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# Early stopping callback



early_stopping = callbacks.EarlyStopping(
    monitor='accuracy',  # Monitor validation loss
    patience=10,  # Stop training after 5 epochs without improvement
    restore_best_weights=True  # Restore weights of the best epoch
)

lr_callback = ReduceLROnPlateau(monitor='accuracy', factor=0.5, patience=5,
                                        verbose=1, mode='auto', min_delta=0, cooldown=0, min_lr=0)

#train_generator = create_generators(train_data_split, train_label_split , validation_data_split,validation_label_split, 100,batch_size=32)
train_generator, val_generator, test_generator = create_generators(train_data,
                                                                    train_labels ,
                                                                      val_data,val_labels,
                                                                          test_data,
                                                                              test_labels,
                                                                                  100,
                                                                                      batch_size=32)


history = model.fit(train_generator, validation_data=val_generator,
                    steps_per_epoch=int((len(train_data)*16*16)/32),
                    validation_steps=100,
                      epochs=100,
                      callbacks=[early_stopping, lr_callback],
                      verbose=1)




ValueError: A KerasTensor is symbolic: it's a placeholder for a shape an a dtype. It doesn't have any actual numerical value. You cannot convert it to a NumPy array.

# Predictions

In [ ]:
results = model.evaluate(test_generator, steps=100, verbose=1)
print(f"Test Loss: {results[0]}, Test Accuracy: {results[1]}")

100/100 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.8860 - loss: 0.2928
Test Loss: 0.29363226890563965, Test Accuracy: 0.8890625238418579


In [ ]:
TEST_SIZE = 50

In [ ]:
createSubmission(model, WINDOW_SIZE)

46/46 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
46/46 ━━━━━━━━━━━━━━━━━━